In [1]:
%%capture
!pip install category-encoders
!pip install pyldavis

In [2]:
# import libraries
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import joblib
import spacy
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models
from category_encoders import OneHotEncoder
import numpy as np

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping


In [3]:
# run once
# !python -m spacy download en_core_web_lg

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# load spacy model
nlp = spacy.load("en_core_web_lg")

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Baseline Nearest Neighbors Model

In [5]:
# read in data
data_path = "../data/"

def wrangle(data_path):

    # reads in the data
    df_tracks = pd.read_csv(data_path + 'tracks.csv', parse_dates=['release_date'])
    df_lyrics = pd.read_csv(data_path + 'lyrics.csv') #usecols=['SName', 'Lyric']
    
    # drop null values on df_tracks
    df_tracks.dropna(inplace=True)
    df_lyrics.dropna(inplace=True)
    
    # case normalization on song names
    df_lyrics['SName'] = df_lyrics['SName'].apply(lambda x: x.lower())
    df_tracks['name'] = df_tracks['name'].apply(lambda x: x.lower())
    
    # clean up df_lyrics data for merging
    df_lyrics = df_lyrics.rename(columns={'ALink': 'Artist', 'SName': 'name'})
    df_lyrics['Artist'] = df_lyrics['Artist'].apply(lambda x: x.rstrip('/').lstrip('/').replace('-', " ").lower())

    # drops duplicate lyrics 
    df_lyrics = df_lyrics.drop_duplicates(subset=['Artist', 'name'])
    
    # clean up df_tracks data for merging
    df_tracks = df_tracks.rename(columns={'artists': 'Artist'})

    df_tracks['Artist'] = df_tracks['Artist'].apply(lambda x: x.lstrip("['").rstrip("']").lower())
    
    # use regex to strip acoustic/remix versions
    strip_post_dash = ' - \S+'
    strip_version = ' version'

    df_tracks['name'] = df_tracks['name'].apply(lambda song_name: re.sub(strip_post_dash, "", song_name))
    df_tracks['name'] = df_tracks['name'].apply(lambda song_name: re.sub(strip_version, "", song_name))
    
    # drop duplicate rows
    df_tracks = df_tracks.drop_duplicates(subset=['Artist', 'name'])
    
    df_merged = pd.merge(df_lyrics, df_tracks, on=['Artist', 'name'], how='inner')
    
    # drop unwanted columns
    df_merged = df_merged.drop(columns=['duration_ms'])
    
    df_merged = df_merged[df_merged['Idiom'] == 'ENGLISH']
    
    # original merge dataframes
    # df_merged = pd.merge(df_tracks, df_lyrics, how='inner', left_on=['name'], right_on='SName')
    
    
    # original drop columns not used in NN model
    # df_merged = df_merged.drop(columns= ['id', 'name', 'artists', 'id_artists', 'release_date', ])
    
    return df_merged

def keep_wanted_columns(df):
    df_dropped = df[['popularity', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'new_topic_name_Existential', 'new_topic_name_Religion',
        'new_topic_name_Gangsta', 'new_topic_name_Poppy','new_topic_name_Love']]
    
    #drop(columns= ['Artist', 'name', 'SLink', 'Lyric', 'Idiom', 'id_artists', 'id', 'release_date'])
    
    return df_dropped

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:31: DeprecationWarning: invalid escape sequence \S
<>:31: DeprecationWarning: invalid escape sequence \S
<ipython-input-5-f5e2b701b702>:31: DeprecationWarning: invalid escape sequence \S
  strip_post_dash = ' - \S+'


In [6]:
df_merged = wrangle(data_path)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df_merged.head()

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Artist,name,SLink,Lyric,Idiom,id,popularity,explicit,id_artists,release_date,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,12 stones,world so cold,/12-stones/world-so-cold.html,"It starts with pain, followed by hate. Fueled ...",ENGLISH,471eQ7hcJ7JdGY1NzMmUeg,55,0,['0DrXhci3WAyo0WJv1RBOG6'],2007-01-01,...,4,-4.949,0,0.0443,0.472000,0.000000,0.360,0.394,186.227,3
1,12 stones,anthem for the underdog,/12-stones/anthem-for-the-underdog.html,You say you know just who I am. But you can't ...,ENGLISH,6FFwt1ea9hJ4MfMQLywahm,59,0,['0DrXhci3WAyo0WJv1RBOG6'],2007-01-01,...,5,-3.424,1,0.0626,0.000651,0.000000,0.339,0.468,93.977,3
2,12 stones,we are one,/12-stones/we-are-one.html,We walk alone. In the unknown. We live to win ...,ENGLISH,4lhqal0Hq63U2wETCeBdG1,58,0,['0DrXhci3WAyo0WJv1RBOG6'],2010-01-01,...,5,-4.041,0,0.1140,0.000119,0.000016,0.233,0.306,127.102,4
3,3 doors down,here without you,/3-doors-down/here-without-you.html,"A hundred days have made me older,. Since the ...",ENGLISH,3NLrRZoMF0Lx6zTlYqeIo4,76,0,['2RTUTCvo6onsAnheUk3aL9'],2002-11-12,...,10,-6.817,0,0.0252,0.049200,0.000000,0.205,0.233,143.994,4
4,3 doors down,when i'm gone,/3-doors-down/when-im-gone.html,There's another world inside of me that you ma...,ENGLISH,3WbphvawbMZ8FyqDxYGdSQ,70,0,['2RTUTCvo6onsAnheUk3aL9'],2002-11-12,...,7,-5.611,1,0.0284,0.003850,0.000000,0.103,0.374,148.095,4


In [8]:
# user selects what song
user_selected_song = 'imagine'
user_selected_artist = 'john lennon'

song_row = df_merged[(df_merged['name'] == user_selected_song) & (df_merged['Artist'] == user_selected_artist)]
song_row

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Artist,name,SLink,Lyric,Idiom,id,popularity,explicit,id_artists,release_date,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
3776,john lennon,imagine,/john-lennon/imagine.html,Imagine there's no heaven. It's easy if you tr...,ENGLISH,4lwh5BoHBFGjKNAF3sNNCK,25,0,['4x1nvY2FN8jxqAFA0DA02H'],1971-09-09,...,0,-11.407,1,0.029,0.947,0.00003,0.0689,0.218,74.4,4


# Natural Language Processing

In [9]:
import re
def clean_data(text):
    """
    Accepts a single text document and performs several regex substitutions in order to clean the document.
    Parameters
    ----------
    text: string or object
    Returns
    -------
    text: string or object
    """
    special_chars_regex = '[:?,\>$|!\'"]'
    white_spaces_regex = '[ ]{2,}'
    text = re.sub('[^a-zA-Z ]', "", text)
    text = re.sub(special_chars_regex, " ", text)
    text = re.sub(white_spaces_regex, " ", text)
    return text.lower()

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:12: DeprecationWarning: invalid escape sequence \>
<>:12: DeprecationWarning: invalid escape sequence \>
<ipython-input-9-2a9d641a4093>:12: DeprecationWarning: invalid escape sequence \>
  special_chars_regex = '[:?,\>$|!\'"]'


In [10]:
def tokenize(doc):
# run the clean_data function on the text
    text = clean_data(doc)
# Instantiatie language model
    doc = nlp(text)
    STOP_WORDS = nlp.Defaults.stop_words.union('', 'the', 'this', '-pron-', 'wo', 've', 'nt')
    return [token.lemma_.lower().lstrip().rstrip() for token in doc
            if (token.text not in STOP_WORDS)
            and (token.is_punct != True)
            and (token.text not in '')
            and (len(token.text)>2)]

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
%%capture
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!pip install pandarallel

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=7)

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
lemmas = df_merged['Lyric'].parallel_apply(tokenize)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
lemmas

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0        [start, pain, follow, hate, fuel, endless, que...
1        [know, imagine, wait, line, think, stand, tire...
2        [walk, unknown, live, win, victory, young, die...
3        [day, old, time, see, pretty, face, thousand, ...
4        [world, inside, secret, life, hide, darkness, ...
                               ...                        
14394    [verse, time, tear, fill, eye, past, shadow, s...
14395    [man, day, leave, say, regret, bond, break, fe...
14441    [stop, sound, good, turn, turn, know, know, st...
15306    [brother, brother, prepare, happy, way, lord, ...
15331    [mama, mama, mama, easy, girl, easy, girl, jor...
Name: Lyric, Length: 12780, dtype: object

In [15]:
# prepare data for LDA model
id2words = corpora.Dictionary(lemmas)
corpus = [id2words.doc2bow(lemma) for lemma in lemmas]

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# train LDA model to generate topics

lda_multicore = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2words,
                                                        num_topics=5,
                                                        chunksize=100,
                                                        passes=10,
                                                        per_word_topics=True,
                                                        workers=6)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
all_topics = lda_multicore.get_document_topics(corpus[3800], per_word_topics=True)
# for doc_topics, word_topics, phi_values in all_topics:
#     print('New Document \n')
#     print('Document topics:', doc_topics)
#     print('Word topics:', word_topics)
#     print('Phi values:', phi_values)
#     print(" ")
#     print('-------------- \n')
all_topics[0]

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(1, 0.3917247), (3, 0.60287964)]

In [18]:
# display the print_topics from the model 
doc_topics = lda_multicore.get_document_topics(corpus, per_word_topics=True)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
len(lemmas[0])

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


130

In [40]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_multicore, corpus, id2words)
vis

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.128341 -0.040133       1        1  34.512948
4     -0.129102 -0.127985       2        1  24.479364
3     -0.074515  0.041710       3        1  20.017332
2      0.040309  0.220551       4        1  17.164511
1      0.291648 -0.094142       5        1   3.825845, topic_info=       Term          Freq         Total Category  logprob  loglift
113    baby  10986.000000  10986.000000  Default  30.0000  30.0000
33     love  21368.000000  21368.000000  Default  29.0000  29.0000
110    yeah  10260.000000  10260.000000  Default  28.0000  28.0000
1216    ooh   1999.000000   1999.000000  Default  27.0000  27.0000
426   wanna   6781.000000   6781.000000  Default  26.0000  26.0000
...     ...           ...           ...      ...      ...      ...
401     new    179.679158   2351.463403   Topic5  -5.5611   0.6918
344    burn    167.905196   2352.387263   Topic5  -5.6288   0.6236
215    star    155.738843   1498.606000   Topic5  -5.7041   0.9993
55    world    158.846660   5266.730442   Topic5  -5.6843  -0.2378
745     hey    145.367654   3402.992074   Topic5  -5.7730   0.1102

[441 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
3660       5  0.986320       aaah
7674       4  0.996468        aah
7594       3  0.983845  afterglow
10275      1  0.978136       ahah
432        1  0.859474    alright
...      ...       ...        ...
112        2  0.137680      young
112        3  0.266134      young
112        4  0.088002      young
112        5  0.326458      young
4442       3  0.990144       zero

[1015 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 3, 2])

In [41]:
# create a dictionary 
# keys - use topic ids from pyLDAvis visualization 
# values - topic names that you create 

vis_topic_name_dict = {1:"Existential", 
                       2:"Love", 
                       3:"Gangsta", 
                       4:"Religion", 
                       5:"Poppy"
                       }

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
def get_topic_id_lookup_dict(vis, vis_topic_name_dict):
    """
    Both the starting index and the ordering of topic ids bewteen the trained LDA model 
    and the viz tool are different. So we need to create a look up dictionary that maps 
    the correct association between topic ids from both sources. 
    """
    # value is order of topic ids accoridng to pyLDAvis tool 
    # key is order of topic ids according to lda model
    model_vis_tool_topic_id_lookup = vis.topic_coordinates.topics.to_dict()
    #return model_vis_tool_topic_id_lookup

    # invert dictionary so that 
    # key is order of topic ids accoridng to pyLDAvis tool 
    # value is order of topic ids according to lda model
    topic_id_lookup =  {v:k for k, v in model_vis_tool_topic_id_lookup.items()}
    
    return {v:vis_topic_name_dict[k]  for k, v in topic_id_lookup.items()}

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
# use get_topic_id_lookup_dict to update vis_topic_name_dict - save result to `topic_name_dict`

topic_name_dict = get_topic_id_lookup_dict(vis, vis_topic_name_dict)


/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
def get_topic_ids_for_docs(lda_model, corpus):
    
    """
    Passes a Bag-of-Words vector into a trained LDA model in order to get the topic id of that document. 
    
    Parameters
    ----------
    lda_model: Gensim object
        Must be a trained model 
        
    corpus: nested lists of tuples, 
        i.e. [[(),(), ..., ()], [(),(), ..., ()], ..., [(),(), ..., ()]]
        
    Returns
    -------
    topic_id_list: list
        Contains topic ids for all document vectors in corpus 
    """
    
    # store topic ids for each document
    doc_topic_ids = []

    # iterature through the bow vectors for each doc
    for doc_bow in corpus:
        
        # store the topic ids for the doc
        topic_ids = []
        # store the topic probabilities for the doc
        topic_probs = []

        # list of tuples
        # each tuple has a topic id and the prob that the doc belongs to that topic 
        topic_id_prob_tuples = lda_multicore.get_document_topics(doc_bow)
        
        # iterate through the topic id/prob pairs 
        for topic_id_prob in topic_id_prob_tuples:
            
            # index for topic id
            topic_id = topic_id_prob[0]
            # index for prob that doc belongs that the corresponding topic
            topic_prob = topic_id_prob[1]

            # store all topic ids for doc
            topic_ids.append(topic_id)
            # store all topic probs for doc
            topic_probs.append(topic_prob)

        # get index for largest prob score
        max_topic_prob_ind = np.argmax(topic_probs)
        # get corresponding topic id
        max_prob_topic_id = topic_ids[max_topic_prob_ind]
        # store topic id that had the highest prob for doc being a memebr of that topic
        doc_topic_ids.append(max_prob_topic_id)
        
    return doc_topic_ids

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
# get the topic id for each doc in the corpus 
doc_topic_ids = get_topic_ids_for_docs(lda_multicore, corpus)

# create a feature for document's topic id
# topic ids from trained LDA model
df_merged["topic_id"] = doc_topic_ids

# iterate through the topic id and use the lookup table to assign each document with a topic name
df_merged["new_topic_name"] = df_merged["topic_id"].apply(lambda topic_id: topic_name_dict[topic_id])


/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
cols = ["Lyric", "new_topic_name"]
df_merged[cols].head()

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Lyric,new_topic_name
0,"It starts with pain, followed by hate. Fueled ...",Love
1,You say you know just who I am. But you can't ...,Existential
2,We walk alone. In the unknown. We live to win ...,Love
3,"A hundred days have made me older,. Since the ...",Existential
4,There's another world inside of me that you ma...,Existential


In [47]:
ohe = OneHotEncoder(use_cat_names=True)

ohe_topics = ohe.fit_transform(df_merged['new_topic_name'])
ohe_topics.head()

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/category_encoders/one_hot.py:208: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify 

,new_topic_name_Love,new_topic_name_Existential,new_topic_name_Gangsta,new_topic_name_Religion,new_topic_name_Poppy
0,1,0,0,0,0
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0


In [48]:
df_merged.shape

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(12780, 24)

In [49]:
df_merged_ohe = pd.concat([df_merged, ohe_topics], axis=1)
df_merged_ohe.head()

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Artist,name,SLink,Lyric,Idiom,id,popularity,explicit,id_artists,release_date,...,valence,tempo,time_signature,topic_id,new_topic_name,new_topic_name_Love,new_topic_name_Existential,new_topic_name_Gangsta,new_topic_name_Religion,new_topic_name_Poppy
0,12 stones,world so cold,/12-stones/world-so-cold.html,"It starts with pain, followed by hate. Fueled ...",ENGLISH,471eQ7hcJ7JdGY1NzMmUeg,55,0,['0DrXhci3WAyo0WJv1RBOG6'],2007-01-01,...,0.394,186.227,3,4,Love,1,0,0,0,0
1,12 stones,anthem for the underdog,/12-stones/anthem-for-the-underdog.html,You say you know just who I am. But you can't ...,ENGLISH,6FFwt1ea9hJ4MfMQLywahm,59,0,['0DrXhci3WAyo0WJv1RBOG6'],2007-01-01,...,0.468,93.977,3,0,Existential,0,1,0,0,0
2,12 stones,we are one,/12-stones/we-are-one.html,We walk alone. In the unknown. We live to win ...,ENGLISH,4lhqal0Hq63U2wETCeBdG1,58,0,['0DrXhci3WAyo0WJv1RBOG6'],2010-01-01,...,0.306,127.102,4,4,Love,1,0,0,0,0
3,3 doors down,here without you,/3-doors-down/here-without-you.html,"A hundred days have made me older,. Since the ...",ENGLISH,3NLrRZoMF0Lx6zTlYqeIo4,76,0,['2RTUTCvo6onsAnheUk3aL9'],2002-11-12,...,0.233,143.994,4,0,Existential,0,1,0,0,0
4,3 doors down,when i'm gone,/3-doors-down/when-im-gone.html,There's another world inside of me that you ma...,ENGLISH,3WbphvawbMZ8FyqDxYGdSQ,70,0,['2RTUTCvo6onsAnheUk3aL9'],2002-11-12,...,0.374,148.095,4,0,Existential,0,1,0,0,0


In [50]:
# user selects what song
user_selected_song = 'imagine'
user_selected_artist = 'john lennon'

song_row = df_merged_ohe[(df_merged_ohe['name'] == user_selected_song) & (df_merged_ohe['Artist'] == user_selected_artist)]
song_row

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Artist,name,SLink,Lyric,Idiom,id,popularity,explicit,id_artists,release_date,...,valence,tempo,time_signature,topic_id,new_topic_name,new_topic_name_Love,new_topic_name_Existential,new_topic_name_Gangsta,new_topic_name_Religion,new_topic_name_Poppy
3776,john lennon,imagine,/john-lennon/imagine.html,Imagine there's no heaven. It's easy if you tr...,ENGLISH,4lwh5BoHBFGjKNAF3sNNCK,25,0,['4x1nvY2FN8jxqAFA0DA02H'],1971-09-09,...,0.218,74.4,4,4,Love,1,0,0,0,0


In [51]:
# drop categorical data, prepare for input to model
song_row = keep_wanted_columns(song_row)
df_dropped = keep_wanted_columns(df_merged_ohe)

# instantiate Nearest Neighbors model
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')

# fit the model
nn.fit(df_dropped)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NearestNeighbors(algorithm='kd_tree', n_neighbors=10)

In [52]:
neigh_dist, neigh_index = nn.kneighbors(song_row)

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
neigh_index = neigh_index[0][1:]
neigh_index

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 2434, 10275,   906,  2536,  1327,  8731, 11773,  2617,  8643])

In [54]:
for index in neigh_index:
    print(df_merged['name'].iloc[index])

my wish came true
let me go
i'm not there
tell me why
girl on my mind
across the bridge where angels dwell
wings of my love
your love's been a long time coming
too scared to run


/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
# pickle trained model to be used in application
joblib.dump(nn, '../data/NearestNeighborModelWithTopics')

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['../data/NearestNeighborModelWithTopics']

In [56]:
# export merged dataframe to csv
df_merged_ohe.to_csv('../data/df_with_topics.csv')

/Users/dylansivori/.local/share/virtualenvs/Spotify-Song-Suggester-AdyWyJ8v/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
